# Analiza atrybutów

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [54]:
DIR_DATA = 'data/v2'

In [55]:
artists = pd.read_json(path_or_buf=f'{DIR_DATA}/artists.jsonl', lines=True)
sessions = pd.read_json(path_or_buf=f'{DIR_DATA}/sessions.jsonl', lines=True)
tracks = pd.read_json(path_or_buf=f'{DIR_DATA}/tracks.jsonl', lines=True)
users = pd.read_json(path_or_buf=f'{DIR_DATA}/users.jsonl', lines=True)

In [ ]:
features = ['play_growth_previous', 'like_growth_previous', 
            'average_duration_ratio', 'explicit', 'danceability', 
            'energy', 'key', 'loudness', 'speechiness', 'acousticness', 
            'instrumentalness', 'liveness', 'valence', 'tempo',
            'average_genres_ratio']

# ==================== #
# play_growth_previous - przyrost sumy odtworzeń w poprzednim tygodniu
# like_growth_previous - przyrost sumy polubień w poprzednim tygodniu
# average_duration_ratio - średni stosunek czasu słuchania do długości utworu
# average_genres_ratio - średni stosunek ilości pokrywających się ulubionych /
## gatunków użytkownika z zadeklarowanymi gatunkami autora utworu do ich ilości
# ===================== #

target = ['play_growth', 'play_growth_next']

Agregacja danych

In [ ]:
['track_id', 'week', 'average_duration_ratio', 'play_growth', 'play_amount', '']